In [ ]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from scipy.io import loadmat
import h5py
import functools

# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data import serialize_example

TEST_NAME = 'p4677'
DATA_DIR = '/misc/projects/criticalstress-ldrd/cwj/public/Lab_AE/data_lab_p4677/'

In [ ]:
# Mech data
with h5py.File(DATA_DIR+TEST_NAME+'_mechdata.hdf5', 'r') as f:
  time_mech = f.get('time')[:]
  mu = f.get('mu')[:]
  t2f = f.get('t2f')[:]
  tfail = f.get('tfail')[:]
print('tfail', tfail)

# AE data info
Syncfile = loadmat(DATA_DIR+'p4677_AcTimeVector_Run2.mat')
acsettings = loadmat(DATA_DIR+'p4677_run2_AE.mat')

acTime = Syncfile['acTime'][0,:]
WFlength = int(acsettings['Nsamples'][0])
acTime_dt = np.mean(acTime[1:]-acTime[:-1])/WFlength
fs = 1.0/acTime_dt

# parameters data
params_data = {}
params_data['sample_freq']        = 1e3
params_data['ae_sample_freq']     = fs

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(24,8), sharex=True)
fig.subplots_adjust(hspace=0.0)

ax[-2].plot(time_mech, mu, 'b-', linewidth=1.0)
ax[-2].set_xlabel('Time [s]')
ax[-2].set_ylabel('Friction coefficient')
ax[-2].grid()
ax[-2].xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.3f'))

ax[-1].plot(time_mech, t2f, 'k-', linewidth=1.0)
ax[-1].plot(tfail, np.zeros_like(tfail), 'ro')
ax[-1].set_xlabel('Time [s]')
ax[-1].set_ylabel('T2F [s]')
ax[-1].grid()
ax[-1].xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.3f'))

In [ ]:
folder_name = DATA_DIR+TEST_NAME+'_data_tfrecord/'
if not os.path.exists(folder_name):
  os.mkdir(folder_name)

# write tfrecord files
num_total_batch = time_mech.shape[0]
num_batch_per_file = 10000
num_files = int(np.ceil(num_total_batch / num_batch_per_file))

for i_file in range(num_files):
  fout = 'ae_waveform_data_{:03d}.tfrecord'.format(i_file)
  print('write file', fout)
  
  # create a writer
  with tf.io.TFRecordWriter(folder_name+fout) as tfrecord_writer:
    idx0 = i_file*num_batch_per_file
    
    # read ae hdf5 file
    DATA_FILE_NAME = DATA_DIR+TEST_NAME+'_ae_data.hdf5'
    with h5py.File(DATA_FILE_NAME, "r") as f:
      WFlength = f['time_seg'].attrs['WFlength']
      numWFpfilepCH = f['time_seg'].attrs['numWFpfilepCH']
      TruenumPointspfilepCH = f['time_seg'].attrs['TruenumPointspfilepCH']
      time_seg = f['time_seg'][:]
        
      for k_batch in range(num_batch_per_file):
        if (idx0 + k_batch) >= num_total_batch: break

        # read ae data
        start_time = time_mech[idx0 + k_batch]
        end_time = start_time + 1.0/params_data['sample_freq']
        time_window = np.array([start_time, end_time], dtype=np.float64)
        
        time_seg_start_ind = np.where((time_seg<time_window[0]))[0][-1]
        time_seg_end_ind = np.where((time_seg>time_window[-1]))[0][0]
        
        start_ind = time_seg_start_ind * WFlength
        end_ind = (time_seg_end_ind+1) * WFlength
        ae = f['ae'][start_ind:end_ind,:]
        time = f['time'][start_ind:end_ind]
        
        start_ind = np.where((time<time_window[0]))[0][-1] + 1
        end_ind = np.where((time>time_window[-1]))[0][0]
        wf_ae = ae[start_ind:end_ind,:]
        time_ae = time[start_ind:end_ind]

        # trim to same size
        wf_ae_size_1 = wf_ae.shape[0]
        wf_ae_size = int(params_data['ae_sample_freq']*1.0/params_data['sample_freq'])
        wf_start_ind = int(wf_ae.shape[0]/2)-int(wf_ae_size/2)
        wf_end_ind = wf_start_ind + wf_ae_size
        wf_ae = wf_ae[wf_start_ind:wf_end_ind]
        time_ae = time_ae[wf_start_ind:wf_end_ind]
        
        # serialize example
        if k_batch%1000 == 0: print('     ', time_window, time_ae[0], time_ae[-1], wf_ae_size_1, wf_ae.shape)
        if k_batch%1000 == 999: print('     ', time_window, time_ae[0], time_ae[-1], wf_ae_size_1, wf_ae.shape, '\n')
        example_proto = serialize_example(wf_ae, time_window)
        tfrecord_writer.write(example_proto)